In [2]:
from scipy.io import loadmat
import matplotlib.pyplot as plt
import numpy as np
import scipy
import pyMMF

%matplotlib inline

SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 18

plt.rcParams.update({'font.size': MEDIUM_SIZE})
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

## 1. Parameters of the fiber
Comsol simulation were done using those same parameters

In [ ]:
NA = 0.2
radius = 25 # in microns
areaSize = 2.4*radius # calculate the field on an area larger than the diameter of the fiber
npoints = 2**7 # resolution of the window
n1 = 1.45
wl = 1.55 # wavelength in microns
curvature = None
k0 = 2.*np.pi/wl


r_max = 3.2*radius
npoints_search = 2**8
dh = 2*radius/npoints_search

## 2. Compute transverse modes using pyMMF's axisymmetric solver

In [ ]:
profile = pyMMF.IndexProfile(npoints = npoints, areaSize = areaSize)
profile.initParabolicGRIN(n1=n1,a=radius,NA=NA)

In [ ]:
plt.figure()
plt.imshow(profile.n.reshape([npoints]*2))
plt.title(r'Index porfile')
plt.axis('off')
plt.colorbar()

In [ ]:
solver = pyMMF.propagationModeSolver()
solver.setIndexProfile(profile)
solver.setWL(wl)
modes = solver.solve(
        mode='radial',
        curvature = None,
        r_max = r_max, # max radius to calculate (and first try for large radial boundary condition)
        dh = dh, # radial resolution during the computation
        min_radius_bc = 1.5, # min large radial boundary condition
        change_bc_radius_step = 0.95, #change of the large radial boundary condition if fails 
        N_beta_coarse = int(1e3) # number of steps of the initial coarse scan
        )

### Get the mode matrix and rearrange to mode order 
(to fit with comsol data later on)

In [ ]:
M0_as = modes.getModeMatrix(npola = 2)
Nmodes = modes.number
new_ind = [i//2 if i%2 == 0 else (i-1)//2+Nmodes for i in range(2*Nmodes)]
M0_as = M0_as[:,new_ind]
betas_as = np.sort(np.concatenate([modes.betas]*2))[::-1]
M = modes.m
L = modes.l

## 3. Compute transverse modes using pyMMF's 2D finite difference eigenvalue solver
See wavefrontshapin.net tutorial on [solving the Helmholtz discretized equation](https://www.wavefrontshaping.net/post/id/3) and on [pyMMF implementation](https://www.wavefrontshaping.net/post/id/6).

In [ ]:
modes = solver.solve(
        mode='eig',
        nmodesMax=Nmodes)

In [ ]:
M0_eig = modes.getModeMatrix(npola = 2)
M0_eig = M0_eig[:,new_ind]
# dupliate the propagation constants for the two polarizations
betas_eig = np.abs(np.sort(np.concatenate([modes.betas]*2))[::-1])

## 4. WKB approximation

In [ ]:
n2 = np.sqrt(n1**2-NA**2)
Delta = NA**2/(2.*n2**2)
# b = radius/np.sqrt(2*Delta)
b = radius*n2/NA
f_parabolic = lambda r: np.sqrt(n1**2*(1.-(r/b)**2))

In [ ]:
r_vec = np.linspace(0, 0.7*areaSize, 150)
real_profile = [profile.radialFunc(r) for r in r_vec]
infinite_GRIN_profile = [f_parabolic(r) for r in r_vec]
plt.figure()
plt.plot(r_vec,real_profile)
plt.plot(r_vec,infinite_GRIN_profile, '--')

## propagation constants unde WKB approximation

$\sqrt{k_o^2 n_1^2-2\alpha \left( |l|+2m+1\right)}$

$\alpha = k_o n_1/b$ 

and 

$b = \frac{radius \times n_1}{NA}$

In [ ]:
n0 = np.min(profile.n)
alpha = k0*n2/b
betas_wkb = []

for m,l in zip(M,L):
    betas_wkb.append(np.sqrt(k0**2*n1**2-2*alpha*(np.abs(m)+2*l+1)))
 

betas_wkb = np.concatenate([betas_wkb]*2)
new_ind = np.argsort(betas_wkb)[::-1]
betas_wkb = betas_wkb[new_ind]

### Mode profile under WKB approximation

$\psi_{l,m}(r, \phi) = A e^{- \frac{\alpha r^2}{2}}
(\alpha r^2)^{|m|/2} L_l^{|m|}(\alpha r^2)e^{im\phi}
$

$L_l^{|m|}$ Laguerre polynomial

In [1]:
from scipy.special import genlaguerre
M0_wkb = np.empty(shape = (2*npoints**2,2*Nmodes), dtype = np.complex)
aR2 = alpha*profile.R.ravel()**2

prev_m, prev_l = -1, -1

for i,(m,l) in enumerate(zip(M,L)):
    
    c = -1 if (prev_m == m and prev_l == l) else 1       
    mode_profile = np.exp(-aR2/2)*aR2**(np.abs(m)/2)*genlaguerre(l,np.abs(m))(aR2)*np.exp(1j*c*m*profile.TH.ravel())
    mode_profile = mode_profile/np.linalg.norm(mode_profile)
    M0_wkb[:npoints**2,i] = mode_profile
    M0_wkb[npoints**2:,i+Nmodes] = mode_profile
    
    prev_m = m
    prev_l = l
    
M0_wkb = M0_wkb[...,new_ind]

NameError: name 'np' is not defined

## 5. Load Comsol data
We used the same paramters as for pyMMF calculations.
Simulation are full vectorial FDTD simulation, so we do have a 
(small) longitudinal component of the optical field. 

In [ ]:
data = np.load('modes_transverse_comsol_128.npz')
# matrix of transverse mode profiles
M0_cs = data['M0_cs']
# propagation constats 
betas_cs = data['betas']

### Show one mode

In [14]:
ind = 50

Ex = M0_cs[:npoints**2,ind].reshape([npoints]*2)
Ey = M0_cs[npoints**2:2*npoints**2,ind].reshape([npoints]*2)
Ez = M0_cs[2*npoints**2:,ind].reshape([npoints]*2)

max_E = np.max([np.max(np.abs(E)) for E in [Ex,Ey]])

plt.figure(figsize=[12,5])
plt.subplot(131)
plt.imshow(np.abs(Ex), vmax = max_E)
plt.axis('off')
plt.title(r'$|E_x|$')
plt.subplot(132)
plt.imshow(np.abs(Ey), vmax = max_E)
plt.axis('off')
plt.title(r'$|E_y|$')
plt.subplot(133)
plt.imshow(np.abs(Ez), vmax = max_E)
plt.axis('off')
plt.title(r'$|E_z|$')
plt.suptitle(f'Full vectorial FDTD (Comsol)\n Mode {ind}')

plt.figure(figsize=[8,5])
plt.subplot(121)
plt.imshow(np.sqrt(np.abs(Ex)**2+np.abs(Ey)**2), vmax = max_E)
plt.axis('off')
plt.title(r'$|E_t|$')
plt.subplot(122)
plt.imshow(np.abs(Ey), vmax = max_E)
plt.axis('off')
plt.title(r'$|E_y|$')
plt.suptitle(f'Full vectorial FDTD (Comsol)\n Mode {ind}')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.98, 'Full vectorial FDTD (Comsol)\n Mode 50')

## 6. Compare propagation constants

### 6.1 Whole range

In [15]:
msize = 10
beta_max = k0*n1
beta_min = k0*np.min(profile.n)
plt.figure(figsize = (10,6))
plt.plot(np.abs(betas_cs[:110]),'b+', label = 'Comsol', markersize = msize, markeredgewidth=2)
plt.plot(betas_as,'r*', label = 'pyMMF axisymmetric solver', markersize = msize, markeredgewidth=2)
plt.plot(betas_eig,'gx', label = 'pyMMF 2D eigenvalue solver', markersize = msize, markeredgewidth=2)
plt.plot(betas_wkb,'k.', label = 'WKB', markersize = msize,markeredgewidth=2)
plt.axhline(beta_min, color='g')
plt.axhline(beta_max, color='g')
plt.xlabel('Modes')
plt.ylabel('Propagation constant')
plt.ylim([5.81,5.88])
plt.legend(fontsize=16, loc = 'upper right')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Show a mode


### 6.2 Zoom on lower order modes

In [16]:
msize = 14
xlim = [0,10]
ylim = [5.86,5.875]
beta_max = k0*n1
beta_min = k0*np.min(profile.n)
plt.figure(figsize = (10,6))
plt.plot(np.abs(betas_cs[:110]),'b+', label = 'Comsol', markersize = 1.5*msize, markeredgewidth=2)
plt.plot(betas_as,'r*', label = 'pyMMF axisymmetric solver', markersize = msize, markeredgewidth=2)
plt.plot(betas_eig,'gx', label = 'pyMMF 2D eigenvalue solver', markersize = msize, markeredgewidth=2)
plt.plot(betas_wkb,'k.', label = 'WKB', markersize = msize,)
plt.axhline(beta_min, color='g')
plt.axhline(beta_max, color='g')
plt.xlabel('Modes')
plt.ylabel('Propagation constant')
plt.xlim(xlim)
plt.ylim(ylim)
plt.legend(fontsize=16, loc = 'upper right')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 6.3 Zoom on higher order modes

In [17]:
xlim = [89,110]
ylim = [5.8215,5.824]
beta_max = k0*n1
beta_min = k0*np.min(profile.n)
plt.figure(figsize = (10,6))
plt.plot(np.abs(betas_cs[:110]),'b+', label = 'Comsol', markersize = 1.5*msize, markeredgewidth=2)
plt.plot(betas_as,'r*', label = 'pyMMF axisymmetric solver', markersize = msize, markeredgewidth=2)
plt.plot(betas_eig,'gx', label = 'pyMMF 2D eigenvalue solver', markersize = msize, markeredgewidth=2)
plt.plot(betas_wkb,'k.', label = 'WKB', markersize = msize,)
plt.axhline(beta_min, color='g')
plt.axhline(beta_max, color='g')
plt.xlabel('Modes')
plt.ylabel('Propagation constant')
plt.xlim(xlim)
plt.ylim(ylim)
plt.legend(fontsize=16, loc = 'upper right')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
i = 4
ex = M0_cs[:npoints**2,i].reshape([npoints]*2)
ey = M0_cs[npoints**2:2*npoints**2,i].reshape([npoints]*2)
ex2 = M0_as[:npoints**2,i].reshape([npoints]*2)
ey2 = M0_as[npoints**2:,i].reshape([npoints]*2)

m = np.max([np.max(np.abs(ex)),np.max(np.abs(ey))])
m2 = np.max([np.max(np.abs(ex2)),np.max(np.abs(ey2))])
plt.figure(figsize=(9,9))
plt.subplot(221)
plt.imshow(np.abs(ex), vmax = m)
plt.axis('off')
plt.title(r'$|E_x|$')
plt.subplot(222)
plt.imshow(np.abs(ey), vmax = m)
plt.axis('off')
plt.title(r'$|E_y|$')
plt.subplot(223)
plt.imshow(np.abs(ex2), vmax = m2)
plt.axis('off')
plt.subplot(224)
plt.imshow(np.abs(ey2), vmax = m2)
plt.axis('off')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-0.5, 127.5, 127.5, -0.5)

## 7. Projection of one basis onto the other one
#### The order **inside** each degenerate group is not exacty the same, but it is normal as they are degenerate.

### First compute the mask corresponding to groups of degenerate modes

In [19]:
ind_degenerate = modes.getNearDegenerate(tol = 1e-3)
mask_degenerate = np.zeros(shape = [Nmodes*2]*2, dtype = np.float)

for ind in ind_degenerate:
    mask_degenerate[2*min(ind):2*max(ind)+2,2*min(ind):2*max(ind)+2] = 1

# mask_degenerate = np.kron(np.eye(2),mask_degenerate)  
plt.close('all')
plt.figure()
plt.imshow(mask_degenerate)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 7.1 Display conversion matrices

In [21]:
# only keep transverse field and the propagating modes
M0_cs_trans = M0_cs[:2*npoints**2,:110]

A_as = M0_cs_trans.transpose().conjugate()@M0_as
A_eig = M0_cs_trans.transpose().conjugate()@M0_eig
A_wkb = M0_cs_trans.transpose().conjugate()@M0_wkb

In [ ]:
energy_ratio_as = 1-np.linalg.norm(A_as*mask_degenerate)/np.linalg.norm(A_as)
energy_ratio_eig = 1-np.linalg.norm(A_eig*mask_degenerate)/np.linalg.norm(A_eig)
energy_ratio_wkb = 1-np.linalg.norm(A_wkb*mask_degenerate)/np.linalg.norm(A_wkb)

plt.figure(figsize = (10,4))
plt.subplot(131)
plt.imshow(np.abs(A_as), interpolation = 'None')
plt.axis('off')
plt.title(f'axisymmetric solver\n ratio = {100*energy_ratio_as:.2e}%')
plt.subplot(132)
plt.imshow(np.abs(A_eig), interpolation = 'None')
plt.axis('off')
plt.title(f'2D eigenvalue solver\n ratio = {100*energy_ratio_eig:.2e}%')
plt.subplot(133)
plt.imshow(np.abs(A_wkb), interpolation = 'None')
plt.axis('off')
plt.title(f'WKB\n ratio = {100*energy_ratio_wkb:.2e}%')


### 7.2 Conversion efficiency per mode

For each input mode found using the different technique, 
compute the energy that couples to the corresponding group of degenerate modes
found using Comsol.

In [ ]:
Proj_as = np.sqrt(np.sum( np.abs(A_as*mask_degenerate)**2, axis = 1))
Proj_eig = np.sqrt(np.sum( np.abs(A_eig*mask_degenerate)**2, axis = 1))
Proj_wkb = np.sqrt(np.sum( np.abs(A_wkb*mask_degenerate)**2, axis = 1))

plt.figure()
plt.plot(100*Proj_as, label = 'Assymmetric solver')
plt.plot(100*Proj_eig, label = '2D eigenvalue solver')
plt.plot(100*Proj_wkb, label = 'WKB')
plt.ylabel('Conversion efficiency')
plt.xlabel('Modes')
plt.legend()
yticks = np.linspace(95,100,6)
plt.yticks(yticks, [f'{y:g}%' for y in yticks])
plt.title('Conversion efficiency onto the mode\n basis computed with Comsol')
plt.tight_layout()

### 7.3 Global conversion losses

We compute the singular value decomposition of the conversion matrix between the modes found using Comsol
and using the pyMMF solvers $M_\text{cs}^\dagger . M_\text{pyMMF}$. 
Ideally, if two bases represent the same subspace, the conversion matrix has to be unitary, 
i.e. all its singular values are equal to one.

In [ ]:
_,s_as,_ = np.linalg.svd(A_as)
_,s_eig,_ = np.linalg.svd(A_eig)
_,s_wkb,_ = np.linalg.svd(A_wkb)
plt.figure(figsize = (10,6))
plt.plot(np.sort(np.abs(s_as)), label = 'axisymmetric solver')
plt.plot(np.sort(np.abs(s_eig)), label = '2D eigenvalue solver')
plt.plot(np.sort(np.abs(s_wkb)), label = 'WKB')
plt.ylim([0.5, 1.1])
plt.legend()
plt.title('Singular values of conversion matrices')